# Regression Exercise - Solutions

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [31]:
import pandas as pd

In [32]:
housing = pd.read_csv('cal_housing_clean.csv')

In [33]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [34]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [35]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [36]:
y_val = housing['medianHouseValue']

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [39]:
from sklearn.preprocessing import MinMaxScaler

In [40]:
scaler = MinMaxScaler()

In [41]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [42]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [43]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [44]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [45]:
import tensorflow as tf

In [46]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [47]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [48]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [49]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/28/6fl1s6897q17ww4ypjgd5p540000gn/T/tmpmjnd2_uk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x113511668>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [50]:
model.train(input_fn=input_func,steps=25000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/28/6fl1s6897q17ww4ypjgd5p540000gn/T/tmpmjnd2_uk/model.ckpt.
INFO:tensorflow:loss = 723927960000.0, step = 1
INFO:tensorflow:global_step/sec: 332.572
INFO:tensorflow:loss = 734615800000.0, step = 101 (0.302 sec)
INFO:tensorflow:global_step/sec: 449.873
INFO:tensorflow:loss = 419182600000.0, step = 201 (0.222 sec)
INFO:tensorflow:global_step/sec: 445.091
INFO:tensorflow:loss = 270027200000.0, step = 301 (0.227 sec)
INFO:tensorflow:global_step/sec: 435.203
INFO:tensorflow:loss = 547233100000.0, step = 401 (0.229 sec)
INFO:tensorflow:global_step/sec: 424.439
INFO:tensorflow:loss = 582785760000.0, step = 501 (0.233 sec)
INFO:tensorflow:global_step/sec: 446.504
INFO:tensorflow:loss = 363509970000.0, s

INFO:tensorflow:loss = 137902830000.0, step = 7601 (0.213 sec)
INFO:tensorflow:global_step/sec: 495.751
INFO:tensorflow:loss = 41709134000.0, step = 7701 (0.202 sec)
INFO:tensorflow:global_step/sec: 477.527
INFO:tensorflow:loss = 122826680000.0, step = 7801 (0.209 sec)
INFO:tensorflow:global_step/sec: 463.143
INFO:tensorflow:loss = 165445240000.0, step = 7901 (0.216 sec)
INFO:tensorflow:global_step/sec: 460.558
INFO:tensorflow:loss = 50649620000.0, step = 8001 (0.217 sec)
INFO:tensorflow:global_step/sec: 464.697
INFO:tensorflow:loss = 78797460000.0, step = 8101 (0.215 sec)
INFO:tensorflow:global_step/sec: 496.778
INFO:tensorflow:loss = 196987190000.0, step = 8201 (0.201 sec)
INFO:tensorflow:global_step/sec: 448.89
INFO:tensorflow:loss = 244492350000.0, step = 8301 (0.223 sec)
INFO:tensorflow:global_step/sec: 455.402
INFO:tensorflow:loss = 63667356000.0, step = 8401 (0.221 sec)
INFO:tensorflow:global_step/sec: 456.869
INFO:tensorflow:loss = 107960420000.0, step = 8501 (0.218 sec)
INFO:t

INFO:tensorflow:loss = 23421030000.0, step = 15501 (0.216 sec)
INFO:tensorflow:global_step/sec: 467.239
INFO:tensorflow:loss = 108888470000.0, step = 15601 (0.214 sec)
INFO:tensorflow:global_step/sec: 467.905
INFO:tensorflow:loss = 81769120000.0, step = 15701 (0.214 sec)
INFO:tensorflow:global_step/sec: 442.27
INFO:tensorflow:loss = 146574440000.0, step = 15801 (0.226 sec)
INFO:tensorflow:global_step/sec: 459.749
INFO:tensorflow:loss = 39093363000.0, step = 15901 (0.218 sec)
INFO:tensorflow:global_step/sec: 462.304
INFO:tensorflow:loss = 122001420000.0, step = 16001 (0.216 sec)
INFO:tensorflow:global_step/sec: 463.68
INFO:tensorflow:loss = 58823295000.0, step = 16101 (0.216 sec)
INFO:tensorflow:global_step/sec: 466.397
INFO:tensorflow:loss = 57162682000.0, step = 16201 (0.214 sec)
INFO:tensorflow:global_step/sec: 474.768
INFO:tensorflow:loss = 62225297000.0, step = 16301 (0.211 sec)
INFO:tensorflow:global_step/sec: 478.87
INFO:tensorflow:loss = 101922320000.0, step = 16401 (0.209 sec)


INFO:tensorflow:global_step/sec: 456.565
INFO:tensorflow:loss = 40560894000.0, step = 23401 (0.221 sec)
INFO:tensorflow:global_step/sec: 466.816
INFO:tensorflow:loss = 78723390000.0, step = 23501 (0.214 sec)
INFO:tensorflow:global_step/sec: 439.404
INFO:tensorflow:loss = 65366650000.0, step = 23601 (0.226 sec)
INFO:tensorflow:global_step/sec: 434.958
INFO:tensorflow:loss = 96243925000.0, step = 23701 (0.230 sec)
INFO:tensorflow:global_step/sec: 476.965
INFO:tensorflow:loss = 38338900000.0, step = 23801 (0.210 sec)
INFO:tensorflow:global_step/sec: 473.772
INFO:tensorflow:loss = 54353744000.0, step = 23901 (0.211 sec)
INFO:tensorflow:global_step/sec: 472.561
INFO:tensorflow:loss = 89936140000.0, step = 24001 (0.212 sec)
INFO:tensorflow:global_step/sec: 472.259
INFO:tensorflow:loss = 123741390000.0, step = 24101 (0.212 sec)
INFO:tensorflow:global_step/sec: 482.049
INFO:tensorflow:loss = 112427730000.0, step = 24201 (0.207 sec)
INFO:tensorflow:global_step/sec: 479.635
INFO:tensorflow:loss 

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [51]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [52]:
pred_gen = model.predict(predict_input_func)

In [53]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/28/6fl1s6897q17ww4ypjgd5p540000gn/T/tmpmjnd2_uk/model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Calculate the RMSE. Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [59]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [60]:
from sklearn.metrics import mean_squared_error

In [61]:
mean_squared_error(y_test,final_preds)**0.5

98097.15443308598

# Great Job!